In [27]:
from collections import deque
test = deque(maxlen=2)
state = 1 
test.append(state)
action = 0
done = False
state = 2
test.append(state)
action = 2
done = True
state = 3
for p in range(len(test)):
    print(p)
    print(test[p], action, 0, test[-p + 1], done)  
print(test)


0
1 2 0 2 True
1
2 2 0 1 True
deque([1, 2], maxlen=2)


In [1]:
from nfsp_agent import NFSPDQN

/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1 Physical GPUs, 1 Logical GPUs


2024-05-07 23:22:41.627116: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-07 23:22:41.627144: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-07 23:22:41.627151: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-07 23:22:41.627197: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-07 23:22:41.627221: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
from agent_configs import NFSPDQNConfig, RainbowConfig
from game_configs import MississippiMarblesConfig, LeducHoldemConfig
import tensorflow as tf

config_dict = {
    "activation": "relu",
    "kernel_initializer": "he_normal",
    "loss_function": tf.keras.losses.KLDivergence(),
    "per_epsilon": 0.01,
    "sl_activation": "relu",
    "sl_kernel_initializer": "he_normal",
    "num_players": 2,
    "training_steps": 20000,
    "anticipatory_param": 0.1,
    "min_replay_buffer_size": 16,
    "sl_min_replay_buffer_size": 16,
    "minibatch_size": 16,
    "sl_minibatch_size": 16,
    "replay_interval": 32,
    "sl_replay_buffer_size": 2000000,
    "replay_buffer_size": 200000,
    "transfer_interval": 100,
    "n_step": 1,
}
# config = NFSPConfig(config_dict=config_dict, game_config=MississippiMarblesConfig(), rl_config_type=RainbowConfig)
config = NFSPDQNConfig(config_dict=config_dict, game_config=LeducHoldemConfig(), rl_config_type=RainbowConfig)

Using default adam_epsilon: 1e-06
Using default learning_rate: 0.01
Using default optimizer: <class 'keras.src.optimizers.adam.Adam'>
Using loss_function: <keras.src.losses.KLDivergence object at 0x29bc2a3d0>
Using default training_iterations: 1
Using default num_minibatches: 1
Using minibatch_size: 16
Using replay_buffer_size: 200000
Using min_replay_buffer_size: 16
Using training_steps: 20000
Using activation: relu
Using kernel_initializer: he_normal
Using num_players: 2
Using default adam_epsilon: 1e-06
Using default learning_rate: 0.01
Using default optimizer: <class 'keras.src.optimizers.adam.Adam'>
Using loss_function: <keras.src.losses.KLDivergence object at 0x29bc2a3d0>
Using default training_iterations: 1
Using default num_minibatches: 1
Using minibatch_size: 16
Using replay_buffer_size: 200000
Using min_replay_buffer_size: 16
Using training_steps: 20000
Using activation: relu
Using kernel_initializer: he_normal
Using default width: 128
Using default noisy_sigma: 0.5
Using def

In [3]:
from rainbow.rainbow_agent import RainbowAgent
import custom_gym_envs
import gymnasium as gym
import random
import rlcard
# env = gym.make('custom_gym_envs/MississippiMarbles-v0', render_mode="human", players=2)
env = gym.make('custom_gym_envs/LeducHoldem-v0')
agent = NFSPDQN(env, config, RainbowAgent)

1 Physical GPUs, 1 Logical GPUs
observation_dimensions:  (36,)
num_actions:  4
observation_dimensions:  (36,)
num_actions:  4
observation_dimensions:  (36,)
num_actions:  4


/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


observation_dimensions:  (36,)
num_actions:  4
observation_dimensions:  (36,)
num_actions:  4
observation_dimensions:  (36,)
num_actions:  4
observation_dimensions:  (36,)
num_actions:  4


In [4]:
agent.train()

Q Values  [0.19670127 0.26096598 0.21420218 0.34617208]
Selected Action  3
False
0
None
Game Over!
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q Values  [-0.28663241 -0.10636611  0.06032346  0.065293  ]
Selected Action  3
True
1
[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Game Over!
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Transition 0 is None
Transition 1 is None
Q Values  [0.15291438 0.14113195 0.43122961 0.46269991]
Selected Action  3
False
0
None
Game Over!
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q Values  [-0.28663241 -0.10636611  0.06032346  0.065293  ]
Selected Action  3
True
1
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Game Over!
[1. 0. 0. 0

/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be int8, actual type: float64
  logger.warn(
/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be int8, actual type: float64
  logger.warn(
/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observ

Q Values  [0.15291438 0.14113195 0.43122961 0.46269991]
Selected Action  3
False
0
None
Game Over!
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q Values  [-0.2299653  -0.178693    0.21767008  0.02009   ]
Selected Action  2
True
1
[0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Game Over!
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Transition 0 is None
Transition 1 is None
Q Values  [-0.09114787 -0.03232518  0.10117905  0.24579167]
Selected Action  3
False
0
None
Game Over!
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q Values  [-0.2299653  -0.178693    0.21767008  0.02009   ]
Selected Action  2
True
1
[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Game Over!
[1. 0. 

KeyboardInterrupt: 